## Setup

In [1]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [3]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [4]:
# Set item range table name
table = 'RPC_item_range' + datetime.today().strftime('%Y_%m_%d')

In [5]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table,
                'filter' : "AND item_sub_department_name = '400 RODENT PEST CONTROL' "})

snowflake_tools - 2025-07-07 09:53:01.363857+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...
snowflake_tools - 2025-07-07 09:53:08.948328+10:00 - Saved token with '2025-07-07 13:53:08+10:00' expiry...


## Demographic Segments (AU)

In [6]:
RPC_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

In [7]:
# RPC_ds.head(10)

Overall 400 RPC

In [ ]:
ds = con.read_sql_file_into_pandas(
    file="sql/demographic-segments-overall.sql",
    formatting={
        'start_date': "'2024-06-01'",
        'end_date': "'2025-05-31'",
        'filter' : "item_sub_department_name = '400 RODENT PEST CONTROL'"
    }
)

# Check data
ds[["DEMOGRAPHIC_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

In [8]:
total_summary = RPC_ds.groupby('DEMOGRAPHIC_SEGMENT').agg(TOTAL_SALES_ALL_CLASS=('SEGMENT_SALES', 'first')).reset_index()
total_summary

,DEMOGRAPHIC_SEGMENT,TOTAL_SALES_ALL_CLASS
0,Homeowner older kids,511851.98
1,Homeowner younger kids,877409.75
2,Older homeowner,3843724.49
3,Renter no kids,310016.32
4,Renter with kids,280581.77
5,Unclassifiable,64670.81
6,Younger homeowner,702125.23


Class index

In [9]:
segment_index_summary = RPC_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SALES_INDEX', 'SEGMENT_SALES']]
segment_index_summary

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SALES_INDEX,SEGMENT_SALES
0,500 RODENT POISON,Homeowner older kids,0.937367,511851.98
1,500 RODENT POISON,Homeowner younger kids,0.838635,877409.75
2,500 RODENT POISON,Older homeowner,1.146715,3843724.49
3,500 RODENT POISON,Renter no kids,0.726408,310016.32
4,500 RODENT POISON,Renter with kids,0.701336,280581.77
5,500 RODENT POISON,Unclassifiable,1.130103,64670.81
6,500 RODENT POISON,Younger homeowner,0.921335,702125.23
7,500 RODENT REPELLENT,Homeowner older kids,0.945981,63782.39
8,500 RODENT REPELLENT,Homeowner younger kids,0.975408,126008.45
9,500 RODENT REPELLENT,Older homeowner,0.937121,387861.45


## Commercial Industry Segment (NZ)

In [10]:
# Get data
RPC_is = con.read_sql_file_into_pandas(
    file="sql/5-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': 10, 
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [16]:
RPC_is.head(10)

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_QUANTITY,SEGMENT_CUSTOMERS,TOTAL_SALES,TOTAL_QUANTITY,TOTAL_CUSTOMERS,TOTAL_SEG_SALES,TOTAL_SEG_QUANTITY,...,OVERALL_CUSTOMERS,SEGMENT_SALES_SHARE,SEGMENT_QUANTITY_SHARE,SEGMENT_CUSTOMER_SHARE,OVERALL_SALES_SHARE,OVERALL_QUANTITY_SHARE,OVERALL_CUSTOMER_SHARE,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
0,500 RODENT POISON,Accommodation and Food Services,11979.36,533.0,363,173439.62,8089.0,4358,24786.70,1435.0,...,6776,0.069069,0.065892,0.083295,0.072145,0.070801,0.084120,0.957371,0.930661,0.990193
1,500 RODENT POISON,Carpentry Services,2816.62,143.0,95,173439.62,8089.0,4358,5646.09,332.0,...,6776,0.016240,0.017678,0.021799,0.016434,0.016381,0.022727,0.988202,1.079230,0.959168
2,500 RODENT POISON,Cleaning Services,3699.83,227.0,147,173439.62,8089.0,4358,9102.01,615.0,...,6776,0.021332,0.028063,0.033731,0.026493,0.030343,0.032468,0.805211,0.924840,1.038900
3,500 RODENT POISON,Commercial Builder,5559.59,245.0,159,173439.62,8089.0,4358,12200.06,644.0,...,6776,0.032055,0.030288,0.036485,0.035510,0.031774,0.035567,0.902706,0.953227,1.025810
4,500 RODENT POISON,Electrical Services,7364.10,377.0,268,173439.62,8089.0,4358,16241.78,993.0,...,6776,0.042459,0.046607,0.061496,0.047274,0.048993,0.063902,0.898155,0.951280,0.962349
5,500 RODENT POISON,Landscaper and Gardening Services,6649.08,358.0,251,173439.62,8089.0,4358,14177.59,879.0,...,6776,0.038337,0.044258,0.057595,0.041266,0.043369,0.061246,0.929019,1.020493,0.940388
6,500 RODENT POISON,Manufacturing,27341.71,1080.0,389,173439.62,8089.0,4358,57506.96,3197.0,...,6776,0.157644,0.133515,0.089261,0.167381,0.157736,0.090614,0.941826,0.846442,0.985069
7,500 RODENT POISON,Professional Computer and Scientific Services,50952.77,2531.0,886,173439.62,8089.0,4358,94611.69,5881.0,...,6776,0.293778,0.312894,0.203304,0.275379,0.290162,0.197904,1.066813,1.078343,1.027286
8,500 RODENT POISON,Residential Builder,26770.53,1170.0,817,173439.62,8089.0,4358,53719.95,3046.0,...,6776,0.154351,0.144641,0.187471,0.156359,0.150286,0.193625,0.987158,0.962436,0.968217
9,500 RODENT POISON,Retail and Wholesale Trade,30306.03,1425.0,983,173439.62,8089.0,4358,55575.98,3246.0,...,6776,0.174735,0.176165,0.225562,0.161761,0.160154,0.217828,1.080208,1.099974,1.035505


In [11]:
total_summary = RPC_is.groupby('ITEM_CLASS_NAME').agg(
    TOTAL_SALES_ALL_SEGMENTS=('TOTAL_SALES', 'first'),  # consistent within class
    TOTAL_QUANTITY_ALL_SEGMENTS=('TOTAL_QUANTITY', 'first')
).reset_index()
total_summary

,ITEM_CLASS_NAME,TOTAL_SALES_ALL_SEGMENTS,TOTAL_QUANTITY_ALL_SEGMENTS
0,500 RODENT POISON,173439.62,8089.0
1,500 RODENT REPELLENT,24997.84,550.0
2,500 RODENT TRAP,145131.35,11629.0


In [12]:
segment_index_summary = RPC_is[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SALES_INDEX', 'QUANTITY_INDEX', 'CUSTOMER_INDEX']]
segment_index_summary

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
0,500 RODENT POISON,Accommodation and Food Services,0.957371,0.930661,0.990193
1,500 RODENT POISON,Carpentry Services,0.988202,1.079230,0.959168
2,500 RODENT POISON,Cleaning Services,0.805211,0.924840,1.038900
3,500 RODENT POISON,Commercial Builder,0.902706,0.953227,1.025810
4,500 RODENT POISON,Electrical Services,0.898155,0.951280,0.962349
5,500 RODENT POISON,Landscaper and Gardening Services,0.929019,1.020493,0.940388
6,500 RODENT POISON,Manufacturing,0.941826,0.846442,0.985069
7,500 RODENT POISON,Professional Computer and Scientific Services,1.066813,1.078343,1.027286
8,500 RODENT POISON,Residential Builder,0.987158,0.962436,0.968217
9,500 RODENT POISON,Retail and Wholesale Trade,1.080208,1.099974,1.035505


## Drop item range table

In [13]:
# Drop item range table
con.execute_statement_from_sql_string(
    statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
    formatting={'table_name': table}
    )

# Brand view

In [14]:
overall_brand = con.read_sql_file_into_pandas(
    file="sql/Overall_by_brand.sql",
    formatting={
        "start_date":                 start_date,
        "end_date":                   end_date,
        "sub_dept":                   "'400 RODENT PEST CONTROL'"        
    })

In [15]:
overall_brand

,CUSTOMER_TYPE_CODE,BRAND_NAME,SALES,TOTAL_TRX,TOTAL_UNITS,NUM_ITEMS_PURCHASED,OVERALL_SALES,OVERALL_TRX
0,Consumer,KIWICARE,695580.01,31730,37769.0,12,940093.19,39491
1,Consumer,BIG CHEESE,978429.28,40915,51017.0,26,1250863.29,50286
2,Consumer,CLIX,0.22,1,1.0,1,0.22,1
3,Commercial,KIWICARE,244513.18,7761,9740.0,12,940093.19,39491
4,Consumer,TRAPPED!,303072.71,25195,44826.0,10,382278.67,30189
5,Commercial,NOOSKI,5341.51,337,457.0,3,26334.23,1752
6,Consumer,NO,6960.74,365,367.0,1,8275.91,437
7,Commercial,GOTCHA,11099.77,1278,2066.0,3,63059.13,7913
8,Consumer,NOOSKI,20992.72,1415,1747.0,3,26334.23,1752
9,Commercial,BIG CHEESE,272434.01,9371,12437.0,26,1250863.29,50286
